In [16]:
import sys
import logging
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logger = logging.getLogger()
import tqdm

In [2]:
import gspread
from googleapiclient.discovery import build
from google.oauth2.service_account import Credentials

In [3]:
import pandas as pd

In [33]:
gc = gspread.oauth(http_client=gspread.BackOffHTTPClient)

In [27]:
TGT_FOLDER_ID = "1HtDcsv1nSa-iRyVHjR8YsQZqfDYED9jB"
BASE_SPREAD_SHEET_URL = "https://docs.google.com/spreadsheets/d/1x9EmAuCX3MIPzLbqkFVA0FNT83Dswl1xoiAMMjpc3ZA"

In [28]:
def duplicate_spreadsheet(gc, spread_sheet, new_file_name, folder_id):
    if matches:=gc.list_spreadsheet_files(new_file_name, folder_id):
        logger.info(f"{new_file_name} exists")
        try:
            return gc.open(matches[0]["id"])
        except:
            pass

    new_spread_sheet = gc.copy(
        file_id=spread_sheet.id,
        title=new_file_name,
        folder_id=folder_id,
    )
    
    return new_spread_sheet

In [29]:
def get_base_wsheet(spread_sheet, wsheet_name):
    for ws in spread_sheet.worksheets():
        if ws.title == wsheet_name:
            return ws
    return None

In [30]:
df = pd.read_json("./selected_samples.json")

In [34]:
base_spreadsheet = gc.open_by_url(BASE_SPREAD_SHEET_URL)
new_file_name = "falc_eval.blank"
new_sp = duplicate_spreadsheet(gc, base_spreadsheet, new_file_name, TGT_FOLDER_ID)
gc.list_spreadsheet_files(new_file_name, TGT_FOLDER_ID)
for i, (idx, row) in tqdm.tqdm(enumerate(df.iterrows())):
    source = row["source"]
    pred = row["prediction"]
    try:
        ws = new_sp.duplicate_sheet(
            source_sheet_id=get_base_wsheet(new_sp, "exemple").id,
            insert_sheet_index=i+1,
            new_sheet_name=f"Exemple {i+1}"
        )
        ws.update_acell("a1", idx)
        ws.update_acell("d1", source)
        ws.update_acell("g1", pred)
    except:
        pass
    

INFO:root:falc_eval.blank exists


60it [04:11,  4.19s/it]


{'spreadsheetId': '1T4dPMLkR9VV7c6HqasaQx1g8OCjjo7G_h4bR70RU3nM',
 'updatedRange': "'Exemple 1'!A1",
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}

In [84]:
ws.get("a1")

[['1']]